# Imports

In [4]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output

In [65]:
def load_data():
    train = pd.read_json('train.json')
    test = pd.read_json('test.json')
    #Fill 'na' angles with mode
    train.inc_angle = train.inc_angle.replace('na', 0)
    train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
    test.inc_angle = test.inc_angle.replace('na', 0)
    test.inc_angle = test.inc_angle.astype(float).fillna(0.0)
    return train, test

train, test = load_data()

def color_composite(data):
    w,h = 197,197
    rgb_arrays = []
    for i, row in data.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 / band_2

        r = (band_1 + abs(band_1.min())) / np.max((band_1 + abs(band_1.min())))
        g = (band_2 + abs(band_2.min())) / np.max((band_2 + abs(band_2.min())))
        b = (band_3 + abs(band_3.min())) / np.max((band_3 + abs(band_3.min())))

        rgb = np.dstack((r, g, b))
        #Add in to resize for resnet50 use 197 x 197
        rgb = np.resize(rgb, (w,h)).astype(np.float32)
        rgb_arrays.append(rgb)
    return np.array(rgb_arrays)

rgb_train = color_composite(train)
rgb_test = color_composite(test)


y_train = np.array(train['is_iceberg'])

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(rgb_train, y_train, random_state=420, train_size=0.75)

(1604, 197, 197)


C:\Users\Damygame\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Model

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPool2D
from keras.layers import LeakyReLU
from keras.layers.normalization import BatchNormalization 
from keras.utils import np_utils
from keras.datasets import mnist
import keras

In [31]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=5):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    
    return [es, msave]

callbacks = get_callbacks('./model.hdf5')

In [62]:
def CNN(input_shape=(75, 75, 2)):
    
    def fully_connected(model, n_units, activation, dropout_rate=0):
        model.add(Dense(n_units, activation = activation))
        #model.add(Dropout(dropout_rate))
        return model
    
    def post_conv(model, dropout_rate=0):
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(dropout_rate))
        model.add(MaxPooling2D(pool_size=(2,2)))
        return model

    def convolutional_layer(model, filters, kernel_size, padding = 'valid', input_shape=None, dropout_rate=0):
        if input_shape is None:
            model.add(Conv2D(filters, kernel_size=kernel_size, padding=padding))
        else:
            model.add(Conv2D(filters, kernel_size=kernel_size, padding=padding, input_shape=input_shape))
        return post_conv(model, dropout_rate)
    
    model = Sequential()
    
    model.add(BatchNormalization(input_shape=(197,197,3)))
    model = convolutional_layer(model, 16, kernel_size=(2,2), padding='same', dropout_rate=0.3)
    model = convolutional_layer(model, 32, kernel_size=(2,2), padding='same', dropout_rate=0.3)    
    model = convolutional_layer(model, 64, kernel_size=(2,2), padding='same', dropout_rate=0.3)    
    model = convolutional_layer(model, 64, kernel_size=(2,2), padding='same', dropout_rate=0.3)    
    model = convolutional_layer(model, 64, kernel_size=(2,2), padding='same', dropout_rate=0.3)    
    model = convolutional_layer(model, 128, kernel_size=(2,2), padding='same', dropout_rate=0.3)    

    model.add(Flatten())
    
    model = fully_connected(model, 1, "sigmoid")
    
    return model


model = CNN((75, 75, 2))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_79 (Batc (None, 197, 197, 3)       12        
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 197, 197, 16)      208       
_________________________________________________________________
batch_normalization_80 (Batc (None, 197, 197, 16)      64        
_________________________________________________________________
leaky_re_lu_76 (LeakyReLU)   (None, 197, 197, 16)      0         
_________________________________________________________________
dropout_77 (Dropout)         (None, 197, 197, 16)      0         
_________________________________________________________________
max_pooling2d_76 (MaxPooling (None, 98, 98, 16)        0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 98, 98, 32)        2080      
__________

In [ ]:
# without data augmentation
epochs = 1000
batch_size = 32

model.compile(loss='mae', optimizer=keras.optimizers.adam(lr=0.001, decay=5e-4), metrics=['mae'])
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
          callbacks=callbacks, validation_data=[X_valid, y_valid])

print(model.evaluate(X_train, y_train))
print(model.evaluate(X_test, y_test))

# Using Datagen

In [63]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
batch_size = 32
#Lets define the image transormations that we want
gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.1,
                         rotation_range=40)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_one_input(X1, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=420)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]

#Finally create out generator
gen_flow = gen_flow_for_one_input(X_train, y_train)

from keras.callbacks import EarlyStopping, ModelCheckpoint
epochs_to_wait_for_improve = 50

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=epochs_to_wait_for_improve)
checkpoint_callback = ModelCheckpoint('model.h5', monitor='val_loss', 
                                      verbose=1, save_best_only=True, mode='min')

#fit the model
model.fit_generator(gen_flow, validation_data=(X_valid, y_valid), 
                    steps_per_epoch=int(np.ceil(len(X_train)/batch_size)), 
                    epochs=500, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])

ValueError: Error when checking input: expected batch_normalization_79_input to have 4 dimensions, but got array with shape (401, 197, 197)

(1203, 197, 197)

[0.25241673039867457, 0.16393957372945869]
[0.27912027280087598, 0.17202319949321723]


In [550]:
test_predictions = model.predict_generator(test_set, steps = test.shape[0], use_multiprocessing=True)
test_predictions = test_predictions.reshape((test_predictions.shape[0]))
test_predictions = ['{:f}'.format(item) for item in test_predictions]

submission = pd.DataFrame({'id': test["id"], 'is_iceberg': test_predictions})
submission.to_csv('sub172.csv', index = False)
submission.head(10)

,id,is_iceberg
0,5941774d,0.026564
1,4023181e,0.371797
2,b20200e4,0.000140
3,e7f018bb,0.877413
4,4371c8c3,0.042351
5,a8d9b1fd,0.020246
6,29e7727e,0.079988
7,92a51ffb,0.969216
8,c769ac97,0.010322
9,aee0547d,0.000406
